In [51]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import xml.etree.ElementTree as ET
# from lxml import etree
from scipy.sparse import hstack
import numpy as np
import warnings
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
from nltk import word_tokenize


In [75]:
#xml parser
def get_list(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            if type(sent.find('text').text) is str:
                text_string= text_string+ " "+ sent.find('text').text
        text_list.append(text_string)
        for opinion in review.findall('./sentences/sentence/Opinions/Opinion'):
            opinion_dict = {
                opinion.get('category').replace('#','_'): opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list

In [76]:
text_list = get_list('reviews.xml')

In [77]:
#Selecting only 20 most common aspect.
def get_most_common_aspect(opinion_list):
    import nltk
    opinion= []
    for inner_list in opinion_list:
        for _dict in inner_list:
            for key in _dict:
                opinion.append(key)
    most_common_aspect = [k for k,v in nltk.FreqDist(opinion).most_common(6)]
    value = [v for k,v in nltk.FreqDist(opinion).most_common(6)]
#     print(value)
        
    return most_common_aspect

In [78]:
most_common_aspect= get_most_common_aspect(get_list('reviews.xml')[1])
print(most_common_aspect)

['FOOD_QUALITY', 'SERVICE_GENERAL', 'RESTAURANT_GENERAL', 'FOOD_PRICE', 'FOOD_STYLE_OPTIONS', 'AMBIENCE_GENERAL']


In [79]:
text_list[1]

[[{'RESTAURANT_GENERAL': 'negative'},
  {'SERVICE_GENERAL': 'negative'},
  {'SERVICE_GENERAL': 'negative'},
  {'FOOD_QUALITY': 'negative'},
  {'FOOD_STYLE_OPTIONS': 'negative'},
  {'SERVICE_GENERAL': 'negative'},
  {'RESTAURANT_GENERAL': 'negative'}],
 [{'FOOD_QUALITY': 'positive'},
  {'RESTAURANT_GENERAL': 'positive'},
  {'FOOD_STYLE_OPTIONS': 'positive'},
  {'FOOD_PRICE': 'positive'},
  {'FOOD_PRICE': 'positive'},
  {'FOOD_QUALITY': 'positive'}],
 [{'RESTAURANT_GENERAL': 'negative'},
  {'SERVICE_GENERAL': 'negative'},
  {'SERVICE_GENERAL': 'negative'},
  {'FOOD_QUALITY': 'negative'},
  {'FOOD_QUALITY': 'negative'},
  {'FOOD_QUALITY': 'negative'},
  {'RESTAURANT_GENERAL': 'negative'},
  {'RESTAURANT_GENERAL': 'negative'}],
 [{'RESTAURANT_GENERAL': 'positive'},
  {'SERVICE_GENERAL': 'positive'},
  {'FOOD_QUALITY': 'positive'},
  {'FOOD_PRICE': 'positive'},
  {'RESTAURANT_GENERAL': 'positive'}],
 [{'RESTAURANT_GENERAL': 'positive'},
  {'FOOD_QUALITY': 'positive'},
  {'SERVICE_GENERAL': 

In [80]:
#generate data frame
def get_data_frame(text_list,opinion_list,most_common_aspect):
    data={'Review':text_list}
    f = pd.DataFrame(data)
    if opinion_list:
        for inner_list in opinion_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspect:
                        f.loc[opinion_list.index(inner_list),key]=_dict[key]
    return f

In [81]:
root = get_data_frame(text_list[0], text_list[1], most_common_aspect)
root

,Review,RESTAURANT_GENERAL,SERVICE_GENERAL,FOOD_QUALITY,FOOD_STYLE_OPTIONS,FOOD_PRICE,AMBIENCE_GENERAL
0,"Đánh giá từ bài trước, chỗ này từng là một ch...",negative,negative,negative,negative,NaN,NaN
1,"Tôi đã ăn ở Saul nhiều lần, thức ăn luôn tuyệ...",positive,NaN,positive,positive,positive,NaN
2,Tôi rất thất vọng về nhà hàng này. Tôi yêu cầ...,negative,negative,negative,NaN,NaN,NaN
3,"Đi 3 ngày, tôi rất tự hào vì đây là chuyến đi...",positive,positive,positive,NaN,positive,NaN
4,"Mọi lần tôi đến New York, tôi đều phải đến nh...",positive,positive,positive,NaN,NaN,positive
5,"Khi nhìn thấy chỗ này, tôi tự hứa sẽ đến thử ...",negative,NaN,positive,positive,NaN,negative
6,"Danh sách của họ rất phong phú, nhưng chúng t...",NaN,positive,positive,positive,NaN,NaN
7,"Tôi đã ăn ở đây một vài lần, không có lý do g...",negative,negative,positive,NaN,NaN,NaN
8,Nhà hàng nên đóng cửa từ lâu. Chúng tôi đến v...,negative,negative,negative,negative,NaN,NaN
9,Đó là một món pizza ngon. Lớp vỏ pizza được n...,NaN,NaN,positive,positive,NaN,NaN


In [82]:
#generate data frame for aspect extraction task
def get_aspect_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
#         print(df[common_aspect])
        df[common_aspect]=df[common_aspect].replace(['positive','negative'],[1,1])
    df = df.fillna(0)
    return df

In [83]:
# get_data_frame(text_list[0], text_list[1], most_common_aspect)
df_aspect = get_aspect_data_frame(get_data_frame(text_list[0], text_list[1], most_common_aspect), most_common_aspect)
df_aspect

,Review,RESTAURANT_GENERAL,SERVICE_GENERAL,FOOD_QUALITY,FOOD_STYLE_OPTIONS,FOOD_PRICE,AMBIENCE_GENERAL
0,"Đánh giá từ bài trước, chỗ này từng là một ch...",1.0,1.0,1.0,1.0,0.0,0.0
1,"Tôi đã ăn ở Saul nhiều lần, thức ăn luôn tuyệ...",1.0,0.0,1.0,1.0,1.0,0.0
2,Tôi rất thất vọng về nhà hàng này. Tôi yêu cầ...,1.0,1.0,1.0,0.0,0.0,0.0
3,"Đi 3 ngày, tôi rất tự hào vì đây là chuyến đi...",1.0,1.0,1.0,0.0,1.0,0.0
4,"Mọi lần tôi đến New York, tôi đều phải đến nh...",1.0,1.0,1.0,0.0,0.0,1.0
5,"Khi nhìn thấy chỗ này, tôi tự hứa sẽ đến thử ...",1.0,0.0,1.0,1.0,0.0,1.0
6,"Danh sách của họ rất phong phú, nhưng chúng t...",0.0,1.0,1.0,1.0,0.0,0.0
7,"Tôi đã ăn ở đây một vài lần, không có lý do g...",1.0,1.0,1.0,0.0,0.0,0.0
8,Nhà hàng nên đóng cửa từ lâu. Chúng tôi đến v...,1.0,1.0,1.0,1.0,0.0,0.0
9,Đó là một món pizza ngon. Lớp vỏ pizza được n...,0.0,0.0,1.0,1.0,0.0,0.0


In [84]:
df_aspect.reindex(sorted(df_aspect.columns), axis=1)

,AMBIENCE_GENERAL,FOOD_PRICE,FOOD_QUALITY,FOOD_STYLE_OPTIONS,RESTAURANT_GENERAL,Review,SERVICE_GENERAL
0,0.0,0.0,1.0,1.0,1.0,"Đánh giá từ bài trước, chỗ này từng là một ch...",1.0
1,0.0,1.0,1.0,1.0,1.0,"Tôi đã ăn ở Saul nhiều lần, thức ăn luôn tuyệ...",0.0
2,0.0,0.0,1.0,0.0,1.0,Tôi rất thất vọng về nhà hàng này. Tôi yêu cầ...,1.0
3,0.0,1.0,1.0,0.0,1.0,"Đi 3 ngày, tôi rất tự hào vì đây là chuyến đi...",1.0
4,1.0,0.0,1.0,0.0,1.0,"Mọi lần tôi đến New York, tôi đều phải đến nh...",1.0
5,1.0,0.0,1.0,1.0,1.0,"Khi nhìn thấy chỗ này, tôi tự hứa sẽ đến thử ...",0.0
6,0.0,0.0,1.0,1.0,0.0,"Danh sách của họ rất phong phú, nhưng chúng t...",1.0
7,0.0,0.0,1.0,0.0,1.0,"Tôi đã ăn ở đây một vài lần, không có lý do g...",1.0
8,0.0,0.0,1.0,1.0,1.0,Nhà hàng nên đóng cửa từ lâu. Chúng tôi đến v...,1.0
9,0.0,0.0,1.0,1.0,0.0,Đó là một món pizza ngon. Lớp vỏ pizza được n...,0.0


In [85]:
df_aspect

,Review,RESTAURANT_GENERAL,SERVICE_GENERAL,FOOD_QUALITY,FOOD_STYLE_OPTIONS,FOOD_PRICE,AMBIENCE_GENERAL
0,"Đánh giá từ bài trước, chỗ này từng là một ch...",1.0,1.0,1.0,1.0,0.0,0.0
1,"Tôi đã ăn ở Saul nhiều lần, thức ăn luôn tuyệ...",1.0,0.0,1.0,1.0,1.0,0.0
2,Tôi rất thất vọng về nhà hàng này. Tôi yêu cầ...,1.0,1.0,1.0,0.0,0.0,0.0
3,"Đi 3 ngày, tôi rất tự hào vì đây là chuyến đi...",1.0,1.0,1.0,0.0,1.0,0.0
4,"Mọi lần tôi đến New York, tôi đều phải đến nh...",1.0,1.0,1.0,0.0,0.0,1.0
5,"Khi nhìn thấy chỗ này, tôi tự hứa sẽ đến thử ...",1.0,0.0,1.0,1.0,0.0,1.0
6,"Danh sách của họ rất phong phú, nhưng chúng t...",0.0,1.0,1.0,1.0,0.0,0.0
7,"Tôi đã ăn ở đây một vài lần, không có lý do g...",1.0,1.0,1.0,0.0,0.0,0.0
8,Nhà hàng nên đóng cửa từ lâu. Chúng tôi đến v...,1.0,1.0,1.0,1.0,0.0,0.0
9,Đó là một món pizza ngon. Lớp vỏ pizza được n...,0.0,0.0,1.0,1.0,0.0,0.0


In [86]:
def get_positive_data_frame(df1,most_common_aspect):
    for common_aspect in most_common_aspect:
        print(df1[common_aspect])
        df1[common_aspect]=df1[common_aspect].replace(['positive','negative'],[1,0])
        print(df1[common_aspect])
    df1 = df1.fillna(0)
    return df1

In [87]:
root.head()
df_positive = get_positive_data_frame(get_data_frame(text_list[0], text_list[1], most_common_aspect), most_common_aspect)
df_positive

0      negative
1      positive
2      negative
3      positive
4      positive
5      positive
6      positive
7      positive
8      negative
9      positive
10     positive
11     positive
12     positive
13     positive
14     positive
15     positive
16     positive
17     positive
18     positive
19     positive
20     positive
21     positive
22     positive
23     negative
24     positive
25     negative
26     negative
27     negative
28     positive
29     positive
         ...   
101    positive
102    positive
103         NaN
104    positive
105    positive
106    negative
107    positive
108    negative
109         NaN
110    negative
111         NaN
112    negative
113    positive
114         NaN
115         NaN
116    positive
117    negative
118         NaN
119    negative
120         NaN
121    negative
122    negative
123    negative
124         NaN
125    negative
126    positive
127    negative
128         NaN
129    positive
130    positive
Name: FOOD_QUALITY, Leng

,Review,RESTAURANT_GENERAL,SERVICE_GENERAL,FOOD_QUALITY,FOOD_STYLE_OPTIONS,FOOD_PRICE,AMBIENCE_GENERAL
0,"Đánh giá từ bài trước, chỗ này từng là một ch...",0.0,0.0,0.0,0.0,0.0,0.0
1,"Tôi đã ăn ở Saul nhiều lần, thức ăn luôn tuyệ...",1.0,0.0,1.0,1.0,1.0,0.0
2,Tôi rất thất vọng về nhà hàng này. Tôi yêu cầ...,0.0,0.0,0.0,0.0,0.0,0.0
3,"Đi 3 ngày, tôi rất tự hào vì đây là chuyến đi...",1.0,1.0,1.0,0.0,1.0,0.0
4,"Mọi lần tôi đến New York, tôi đều phải đến nh...",1.0,1.0,1.0,0.0,0.0,1.0
5,"Khi nhìn thấy chỗ này, tôi tự hứa sẽ đến thử ...",0.0,0.0,1.0,1.0,0.0,0.0
6,"Danh sách của họ rất phong phú, nhưng chúng t...",0.0,1.0,1.0,1.0,0.0,0.0
7,"Tôi đã ăn ở đây một vài lần, không có lý do g...",0.0,0.0,1.0,0.0,0.0,0.0
8,Nhà hàng nên đóng cửa từ lâu. Chúng tôi đến v...,0.0,0.0,0.0,0.0,0.0,0.0
9,Đó là một món pizza ngon. Lớp vỏ pizza được n...,0.0,0.0,1.0,1.0,0.0,0.0


In [88]:
def get_negative_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['negative','positive'],[1,0])
    df = df.fillna(0)
    return df

In [89]:
get_negative_data_frame(get_data_frame(text_list[0], text_list[1], most_common_aspect), most_common_aspect)

,Review,RESTAURANT_GENERAL,SERVICE_GENERAL,FOOD_QUALITY,FOOD_STYLE_OPTIONS,FOOD_PRICE,AMBIENCE_GENERAL
0,"Đánh giá từ bài trước, chỗ này từng là một ch...",1.0,1.0,1.0,1.0,0.0,0.0
1,"Tôi đã ăn ở Saul nhiều lần, thức ăn luôn tuyệ...",0.0,0.0,0.0,0.0,0.0,0.0
2,Tôi rất thất vọng về nhà hàng này. Tôi yêu cầ...,1.0,1.0,1.0,0.0,0.0,0.0
3,"Đi 3 ngày, tôi rất tự hào vì đây là chuyến đi...",0.0,0.0,0.0,0.0,0.0,0.0
4,"Mọi lần tôi đến New York, tôi đều phải đến nh...",0.0,0.0,0.0,0.0,0.0,0.0
5,"Khi nhìn thấy chỗ này, tôi tự hứa sẽ đến thử ...",1.0,0.0,0.0,0.0,0.0,1.0
6,"Danh sách của họ rất phong phú, nhưng chúng t...",0.0,0.0,0.0,0.0,0.0,0.0
7,"Tôi đã ăn ở đây một vài lần, không có lý do g...",1.0,1.0,0.0,0.0,0.0,0.0
8,Nhà hàng nên đóng cửa từ lâu. Chúng tôi đến v...,1.0,1.0,1.0,1.0,0.0,0.0
9,Đó là một món pizza ngon. Lớp vỏ pizza được n...,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
def tag_text(text):
    from pyvi import ViTokenizer, ViPosTagger
    a = ViPosTagger.postagging(ViTokenizer.tokenize(text))
    return a 
# [('Truong','N'),('dai hoc', 'N')]
def tagging(tuple_tag):
    dict_tag = []
    for i in range(len(tuple_tag[0])):
        t = (tuple_tag[0][i], tuple_tag[1][i])
        dict_tag.append(t)
    return dict_tag

In [91]:
# input: array(review)
# output: array({"word":"tag"})
from pyvi import ViTokenizer, ViPosTagger
def posTag(review):
    tagged_text_list=[]
    for text in review:
        print(tagging(tag_text(text)))
        tagged_text_list.append(tagging(tag_text(text)))
    return tagged_text_list


In [92]:
#Filter the word with tag- noun,adjective,verb,adverb
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
#         print(text_list)
        for t in text_list:
            if t[1] not in ['F','X','Nu', 'P', 'C', 'E', 'I']: 
                final_text.append(t[0])
        final_text_list.append(' '.join(final_text))
    print(final_text_list)
    return final_text_list

In [93]:
def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect

In [94]:
#Stage 1:
#Making list to train
train_text_list,train_opinion_list = get_list('reviews.xml')
most_common_aspect = get_most_common_aspect(train_opinion_list)

In [95]:
tagged_text_list_train=posTag(train_text_list)
joblib.dump(tagged_text_list_train, 'tagged_text_list_train.pkl')
tagged_text_list_train=joblib.load('tagged_text_list_train.pkl')

[('Đánh_giá', 'V'), ('từ', 'E'), ('bài', 'N'), ('trước', 'A'), (',', 'F'), ('chỗ', 'N'), ('này', 'P'), ('từng', 'R'), ('là', 'V'), ('một', 'M'), ('chỗ', 'N'), ('tốt', 'A'), ('nhưng', 'C'), ('giờ', 'N'), ('không', 'R'), ('còn', 'V'), ('nữa', 'R'), ('.', 'F'), ('Chúng_tôi', 'P'), ('đến', 'V'), ('vào', 'E'), ('chiều', 'N'), (',', 'F'), ('chỗ', 'N'), ('này', 'P'), ('trống_trơn', 'A'), (',', 'F'), ('nhân_viên', 'N'), ('thể_hiện', 'V'), ('như', 'C'), ('là', 'V'), ('chúng_tôi', 'P'), ('phá', 'V'), ('họ', 'P'), ('và', 'C'), ('họ', 'P'), ('rất', 'R'), ('bực_mình', 'A'), ('.', 'F'), ('Họ', 'P'), ('không', 'R'), ('đưa', 'V'), ('cho', 'E'), ('chúng_tôi', 'P'), ('mì', 'N'), ('tử_tế', 'A'), (',', 'F'), ('bỏ_qua', 'V'), ('các', 'L'), ('yêu_cầu', 'N'), ('từ', 'E'), ('chúng_tôi', 'P'), (',', 'F'), ('thả', 'V'), ('đĩa', 'N'), ('chúng_tôi', 'P'), ('lên', 'V'), ('bài', 'N'), ('.', 'F'), ('Thức_ăn_tệ', 'N'), ('-', 'F'), ('quá', 'R'), ('ngọt', 'A'), ('hoặc', 'C'), ('quá', 'R'), ('mặn', 'A'), ('và', 'C'), ('

[('Khi', 'N'), ('bạn', 'N'), ('bước', 'V'), ('chân', 'N'), ('vào', 'E'), ('Cosette', 'Np'), (',', 'F'), ('mọi', 'L'), ('thứ', 'N'), ('thật', 'R'), ('kỳ_diệu', 'A'), ('trong', 'E'), ('một', 'M'), ('con', 'Nc'), ('đường', 'N'), ('nhỏ', 'A'), ('ở', 'E'), ('Paris', 'Np'), ('.', 'F'), ('Nhà_hàng', 'N'), ('nhỏ', 'A'), (',', 'F'), ('ấm_áp', 'A'), ('với', 'E'), ('sự', 'Nc'), ('tinh_tế', 'A'), ('của', 'E'), ('Pháp', 'Np'), ('.', 'F'), ('Thức_ăn', 'V'), ('từ', 'E'), ('mức', 'N'), ('trung_bình', 'A'), ('đến', 'V'), ('trên', 'E'), ('trung_bình', 'N'), (';', 'F'), ('Món', 'Nc'), ('soup', 'N'), ('hành', 'V'), ('Pháp', 'Np'), ('không', 'R'), ('ấn_tượng', 'A'), ('và', 'C'), ('món', 'N'), ('tráng_miệng', 'V'), ('nhạt', 'A'), ('nhòa', 'V'), ('.', 'F')]
[('Thật', 'R'), ('kinh_tởm', 'Nc'), ('Pizza', 'Np'), ('giao', 'V'), ('nguội_lạnh', 'N'), ('và', 'C'), ('phô_mai', 'N'), ('thì', 'C'), ('không', 'R'), ('tan', 'V'), ('chảy', 'V'), ('hết', 'T'), ('!', 'F'), ('Nó', 'P'), ('giống', 'A'), ('như', 'C'), ('phô_m

[('Dĩa', 'N'), ('cơm', 'N'), ('mang', 'V'), ('ra', 'R'), ('là', 'C'), ('đã', 'R'), ('thấy', 'V'), ('hấp_dẫn', 'V'), ('rồi', 'R'), ('.', 'F'), ('Ăn', 'V'), ('muỗng', 'N'), ('đầu_tiên', 'A'), ('là', 'V'), ('thấy', 'V'), ('ngon', 'A'), ('quá', 'R'), ('trời', 'N'), ('quá', 'R'), ('đất', 'N'), ('.', 'F'), ('Vị', 'Np'), ('đậm_đà', 'A'), (',', 'F'), ('ngon', 'A'), ('vừa', 'R'), ('phải', 'V'), ('.', 'F'), ('Tuy_nhiên', 'C'), ('giá', 'N'), ('1', 'M'), ('dĩa', 'N'), ('khá', 'R'), ('cao', 'A'), (',', 'F'), ('nhưng', 'C'), ('xứng_đáng', 'A'), ('với', 'E'), ('chất_lượng', 'N'), ('.', 'F'), ('Quán', 'N'), ('sạch_sẽ', 'A'), ('nữa', 'R'), (',', 'F'), ('đáng', 'V'), ('tin_cậy', 'V'), ('về', 'E'), ('an_toàn', 'A'), ('thực_phẩm', 'N'), ('.', 'F')]
[('Cơm', 'N'), ('khá', 'R'), ('ok', 'A'), ('và', 'C'), ('nhiều', 'A'), ('thịt', 'N'), ('nhưng', 'C'), ('mình', 'P'), ('thấy', 'V'), ('vẫn', 'R'), ('quá', 'R'), ('đắt', 'A'), (',', 'F'), ('ngay', 'R'), ('cả', 'T'), ('kim', 'N'), ('chi', 'P'), ('cũng', 'R'), ('tí

[('Nghe', 'V'), ('danh_tiếng', 'N'), ('lâu_đời', 'A'), (',', 'F'), ('sáng', 'N'), ('đấy', 'P'), ('mình', 'P'), ('cũng', 'R'), ('đến', 'V'), ('ăn', 'V'), ('thử', 'V'), (',', 'F'), ('một', 'M'), ('hộp', 'N'), ('15k', 'M'), (',', 'F'), ('lúc', 'N'), ('đấy', 'P'), ('mình', 'P'), ('thấy', 'V'), ('hơi', 'R'), ('mắc', 'A'), (',', 'F'), ('nhưng', 'C'), ('mong', 'V'), ('là', 'V'), ('đáng', 'V'), ('đồng_tiền', 'N'), (',', 'F'), ('nhưng', 'C'), ('ôi_thôi', 'N'), ('khi', 'N'), ('mở', 'V'), ('hộp', 'N'), ('xôi', 'N'), ('ra', 'V'), ('...', 'F'), ('NGUỘI_NGẮT', 'Ny'), (',', 'F'), ('xôi', 'N'), ('nguội', 'A'), (',', 'F'), ('pate', 'N'), ('lạnh', 'A'), (',', 'F'), ('trứng', 'N'), ('cút', 'N'), ('có', 'V'), ('mùi', 'N'), ('hơi', 'N'), ('thiu', 'A'), (',', 'F'), ('thịt', 'N'), ('có', 'V'), ('mùi', 'N'), ('lạ', 'A'), ('...', 'F'), ('.', 'F'), ('Mình', 'P'), ('nhắm_mắt', 'V'), ('ráng', 'V'), ('nhai', 'V'), ('hết', 'R'), (',', 'F'), ('tại_sao', 'X'), ('xôi', 'N'), ('mà_lại', 'V'), ('để', 'E'), ('nguội', 'V'

In [96]:
#train list after filter
final_train_text_list=filterTag(tagged_text_list_train)

['Đánh_giá bài trước chỗ từng là một chỗ tốt giờ không còn nữa đến chiều chỗ trống_trơn nhân_viên thể_hiện là phá rất bực_mình không đưa mì tử_tế bỏ_qua các yêu_cầu thả đĩa lên bài Thức_ăn_tệ quá ngọt quá mặn rất ít kêu tiền tip ít Tránh xa chỗ', 'đã ăn_ở Saul nhiều lần thức_ăn luôn tuyệt_vời Saul là nhà_hàng tốt nhất Smith_Street and Brooklyn Danh_sách rượu luôn ấn_tượng có nhiều giá_trị tốt giá bạn không_thể ăn tốt hơn Manhattan', 'rất thất_vọng nhà_hàng yêu_cầu nhân_viên cơm gói lá_sen nhận được cơm cô đi khỏi yêu_cầu 3 lần khi cô mang món yêu_cầu Đồ ăn tạm ổn không có đặc_biệt Mì khô thịt lợn nhiều dầu hơn bình_thường còn phải ngồi cùng 1 gia_đình ồn_ào không lịch_sự sẽ không trở chỗ', 'Đi 3 ngày rất tự_hào là chuyến đi kết_thúc bỏi tuyệt_vời Dịch_vụ tuyệt_vời giá không_thể đánh_bại Bạn không_thể có sự lựa_chọn sai_lầm', 'Mọi lần đến New_York đều phải đến nhà_hàng Saul đường Smith Mọi người được nấu mức hoàn_hảo dịch_vụ tuyệt_vời trang_trí đặc_sắc thử món gan ngỗng chuyến đi mới nh

In [97]:
#Aspect Based Sentiment analyis of user's input.
# Thái độ nhân viên dở ẹc, thức ăn thì chán nhưng được cái view đẹp
# 'C', 'R', 'F', 'X', 'E', 'I', 
user_input=input("Enter a laptop review:\n\n")
#Preprocessing and vectorizing
tagged_user_input = posTag([user_input])
filter_tagged_user_input = filterTag(tagged_user_input)

print(filter_tagged_user_input)

Enter a laptop review:


[('', 'N')]
['']
['']


In [98]:
#get data frame
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_train_aspect = get_aspect_data_frame(df_train,most_common_aspect)
df_train_aspect = df_train_aspect.reindex(sorted(df_train_aspect.columns), axis=1)

In [99]:
#Similar for test list
test_text_list,test_opinion_list = get_list('test.xml')

In [100]:
tagged_text_list_test=posTag(test_text_list)
joblib.dump(tagged_text_list_test, 'tagged_text_list_test.pkl')
tagged_text_list_test=joblib.load('tagged_text_list_test.pkl')

[('Tôi', 'P'), ('ăn_ở', 'V'), ('đây', 'P'), ('ba', 'M'), ('lần', 'N'), ('và', 'C'), ('luôn', 'R'), ('thấy', 'V'), ('rất', 'R'), ('tuyệt_vời', 'A'), ('.', 'F'), ('Tôi', 'P'), ('đặc_biệt', 'A'), ('đánh_giá', 'V'), ('cao', 'A'), ('nơi', 'N'), ('này', 'P'), ('cho', 'E'), ('một', 'M'), ('dịp', 'N'), ('đặc_biệt', 'A'), ('nào', 'P'), ('đó', 'P'), (',', 'F'), ('dịch_vụ', 'N'), ('sẽ', 'R'), ('rất', 'R'), ('tốt', 'A'), ('.', 'F')]
[('Hãy', 'R'), ('chắc', 'V'), ('rằng', 'C'), ('bạn', 'N'), ('sẽ', 'R'), ('thử', 'V'), ('món', 'Nc'), ('sò', 'N'), ('điệp', 'V'), ('cuộn', 'N'), ('cay', 'A'), ('.', 'F'), ('Nó', 'P'), ('thật', 'R'), ('ngon_miệng', 'A'), ('!', 'F')]
[('Moules', 'N'), ('tuyệt_vời', 'A'), (',', 'F'), ('tôm_hùm', 'N'), ('ravioli', 'N'), ('rất', 'R'), ('mặn', 'A'), ('!', 'F'), ('Đưa', 'V'), ('mẹ', 'N'), ('tối', 'N'), ('tới', 'E'), ('đây', 'P'), ('vào', 'E'), ('ngày', 'N'), ('của', 'E'), ('Mẹ', 'N'), (',', 'F'), ('và', 'C'), ('phục_vụ', 'V'), ('rất', 'R'), ('thô_lỗ', 'A'), ('.', 'F'), ('Anh',

In [101]:
final_test_text_list=filterTag(tagged_text_list_test)


['ăn_ở ba lần luôn thấy rất tuyệt_vời đặc_biệt đánh_giá cao nơi một dịp đặc_biệt dịch_vụ sẽ rất tốt', 'Hãy chắc bạn sẽ thử món sò điệp cuộn cay thật ngon_miệng', 'Moules tuyệt_vời tôm_hùm ravioli rất mặn Đưa mẹ tối ngày Mẹ phục_vụ rất thô_lỗ Anh bảo ngồi vị_trí bất_kỳ khi ngồi anh bảo bàn đã được đặt trước Anh dẫm lên chân lần thứ 2 điều_chỉnh độ sáng đèn Món tráng_miệng bé tí đắt 8', 'Đồ uống luôn được chế_biến tốt giá từng loại rượu_vang rất hợp_lý Thử món đặc_biệt đầu_bếp rất đáng chết thèm Dịch_vụ đáng được 5 sao đánh_giá cao Phòng chờ tầng rất hay_ho', 'Raga s là một nhà_hàng ấm_áp lãng_mạn Các món ăn kỳ_lạ được trình_bày đẹp_mắt là một điểm sáng khi đem sự kết_hợp đầy ngon_miệng cũng có mức giá cực_kỳ tốt Nhân_viên vô_cùng hữu_ích chu_đáo Quầy rượu có đầy những loại bia thú_vị rượu_vang có_giá vừa túi_tiền là một những nhà_hàng ưa_thích không nên bỏ lỡ Hãy giới_thiệu nơi bạn bạn chắc_chắn sẽ cảm_ơn bạn', 'Cứ ngỡ Leon một viên ngọc sáng đã sai phục_vụ bàn đánh rời đồ bẩn áo_khoác 

In [102]:
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)
df_test_aspect = get_aspect_data_frame(df_test,most_common_aspect)
df_test_aspect = df_test_aspect.reindex(sorted(df_test_aspect.columns), axis=1)


In [103]:
#Sort the data frame according to aspect's name and separate data(X) and target(y)
#df_train_aspect = df_train_aspect.sample(frac=1).reset_index(drop=True) #For randoming
X_train= df_train_aspect.Review
y_train = df_train_aspect.drop('Review',1)

#df_test_aspect = df_test_aspect.sample(frac=1).reset_index(drop=True) #For randoming
X_test = df_test_aspect.Review
y_test = df_test_aspect.drop('Review',1)

In [104]:
#Change y_train to numpy array
import numpy as np
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)

In [105]:
#Generate word vecotors using CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_df=1.0)  
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
print(X_test_dtm)

  (0, 1177)	0.4537458171160134
  (0, 1146)	0.26286098176401357
  (0, 1126)	0.23268254146026415
  (0, 1031)	0.1752480570755463
  (0, 987)	0.18059240127712553
  (0, 903)	0.19619055145973682
  (0, 847)	0.19276576701143866
  (0, 799)	0.2579350995893303
  (0, 707)	0.2268729085580067
  (0, 593)	0.1554009716628965
  (0, 515)	0.18059240127712553
  (0, 488)	0.2539452698316746
  (0, 292)	0.3002257771125557
  (0, 291)	0.19980054728433877
  (0, 123)	0.24608105456521992
  (0, 32)	0.319433923119769
  (1, 924)	0.25336948461862097
  (1, 906)	0.2450050549125266
  (1, 847)	0.25336948461862097
  (1, 625)	0.3946138964171961
  (1, 567)	0.17295572994055713
  (1, 375)	0.3590302431074706
  (1, 201)	0.3946138964171961
  (1, 175)	0.3750307988596436
  (1, 125)	0.3750307988596436
  :	:
  (12, 203)	0.367628320427251
  (13, 1207)	0.2413574239312858
  (13, 1180)	0.2413574239312858
  (13, 1127)	0.1768844575312677
  (13, 1119)	0.07667656110352095
  (13, 1011)	0.14764949408168748
  (13, 929)	0.1620739978755135
  (13, 8

In [106]:
#Create various models. These are multi-label models.
nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
C = 1.0 #SVregularization parameter
svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)


/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [107]:
#Predict the test data using classifiers
y_pred_class = nb_classif.predict(X_test_dtm)
y_pred_class_svc = svc.predict(X_test_dtm)
y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
y_pred_class_sgd = sgd.predict(X_test_dtm)

In [108]:
#Following code to test metrics of all aspect extraction classifiers
from sklearn import metrics

In [109]:
print(metrics.accuracy_score(y_test,y_pred_class))
print(metrics.accuracy_score(y_test,y_pred_class_svc))
print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
print(metrics.accuracy_score(y_test,y_pred_class_sgd))

0.14285714285714285
0.14285714285714285
0.21428571428571427
0.07142857142857142


In [110]:
print(metrics.recall_score(y_test,y_pred_class,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))


0.4473684210526316
0.6578947368421053
0.6842105263157895
0.7105263157894737


In [111]:
print(metrics.f1_score(y_test,y_pred_class,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))


0.5573770491803278
0.6944444444444445
0.6842105263157895
0.6506024096385543


In [112]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00         5
           2       0.64      1.00      0.78         9
           3       0.00      0.00      0.00         3
           4       1.00      0.20      0.33         5
           5       0.88      0.78      0.82         9

   micro avg       0.74      0.45      0.56        38
   macro avg       0.42      0.33      0.32        38
weighted avg       0.49      0.45      0.42        38
 samples avg       0.68      0.45      0.53        38

              precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       1.00      0.60      0.75         5
           2       0.64      1.00      0.78         9
           3       0.33      0.33      0.33         3
           4       0.80      0.80      0.80         5
           5       0.88      0.78      0.82         9

   micro avg       0.74

In [113]:
def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect

In [114]:

train_dict_aspect=get_dict_aspect(y_train, most_common_aspect)
d_train=DictVectorizer() 
X_train_aspect_dtm = d_train.fit_transform(train_dict_aspect)

test_dict_aspect=get_dict_aspect(y_test,most_common_aspect)
d_test=DictVectorizer() 
X_test_aspect_dtm = d_test.fit_transform(test_dict_aspect)

In [115]:
#Function for classiflying positive,negative or neutral sentiment of all the aspects
def classify_sentiment(df_train,df_test,X_train_aspect_dtm,X_test_aspect_dtm):
    
    df_train = df_train.reindex_axis(sorted(df_train_positive.columns), axis=1)
    df_test = df_test.reindex_axis(sorted(df_test_positive.columns), axis=1)

    import numpy as np
    X_train = df_train.Review
    y_train = df_train.drop('Review',1)
    y_train = np.asarray(y_train, dtype=np.int64)

    X_test = df_test.Review
    y_test = df_test.drop('Review',1)
    y_test = np.asarray(y_test, dtype=np.int64)

    vect_sen = TfidfVectorizer(ngram_range=(1,2))  
    X_train_dtm = vect_sen.fit_transform(X_train)
    X_test_dtm = vect_sen.transform(X_test)

    #ombining word vector with extra feature.
    from scipy.sparse import hstack
    X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm))

    C = 1.0 #SVregularization parameter
    nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
    svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
    lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
    sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)

    y_pred_class= nb_classif.predict(X_test_dtm)
    y_pred_class_svc = svc.predict(X_test_dtm)
    y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
    y_pred_class_sgd = sgd.predict(X_test_dtm)
    return (y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd)

In [116]:
def print_metrices(y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd):
    print("Accuracy:")
    print(metrics.accuracy_score(y_test,y_pred_class))
    print(metrics.accuracy_score(y_test,y_pred_class_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_sgd))

    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred_class,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred_class,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))
    
    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred_class,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

In [117]:
#For positive sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
df_test_positive = get_positive_data_frame(df_test,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test_positive,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos)

0      negative
1      positive
2      negative
3      positive
4      positive
5      positive
6      positive
7      positive
8      negative
9      positive
10     positive
11     positive
12     positive
13     positive
14     positive
15     positive
16     positive
17     positive
18     positive
19     positive
20     positive
21     positive
22     positive
23     negative
24     positive
25     negative
26     negative
27     negative
28     positive
29     positive
         ...   
101    positive
102    positive
103         NaN
104    positive
105    positive
106    negative
107    positive
108    negative
109         NaN
110    negative
111         NaN
112    negative
113    positive
114         NaN
115         NaN
116    positive
117    negative
118         NaN
119    negative
120         NaN
121    negative
122    negative
123    negative
124         NaN
125    negative
126    positive
127    negative
128         NaN
129    positive
130    positive
Name: FOOD_QUALITY, Leng

/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy:
0.14285714285714285
0.6428571428571429
0.6428571428571429
0.14285714285714285

Average precision:
0.5294117647058824
0.8181818181818182
0.8235294117647058
0.6521739130434783

Average recall:
0.3
0.9
0.9333333333333333
0.5

Average f1:
0.3829787234042553
0.8571428571428572
0.8749999999999999
0.5660377358490566

Classification report:
              precision    recall  f1-score   support

           0       0.50      0.17      0.25         6
           1       0.00      0.00      0.00         4
           2       0.50      1.00      0.67         7
           3       1.00      0.50      0.67         2
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         7

   micro avg       0.53      0.30      0.38        30
   macro avg       0.33      0.28      0.26        30
weighted avg       0.28      0.30      0.25        30
 samples avg       0.54      0.39      0.39        30

              precision    recall  f1-score   support

   

/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [118]:
#For negative sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neg = get_negative_data_frame(df_train,most_common_aspect)
df_test_neg = get_negative_data_frame(df_test,most_common_aspect)

y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_neg,df_test_neg,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg)

/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy:
0.7857142857142857
0.6428571428571429
0.5714285714285714
0.2857142857142857

Average precision:
0.0
0.4
0.2857142857142857
0.2

Average recall:
0.0
0.25
0.25
0.375

Average f1:
0.0
0.3076923076923077
0.26666666666666666
0.26086956521739135

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2

   micro avg       0.00      0.00      0.00         8
   macro avg       0.00      0.00      0.00         8
weighted avg       0.00      0.00      0.00         8
 samples avg       0.00      0.00      0.00         8

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0

/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [ ]:
Thát đo

In [144]:
#Aspect Based Sentiment analyis of user's input.
user_input=input("Enter a review:\n\n")
#Preprocessing and vectorizing
tagged_user_input = posTag([user_input])
filter_tagged_user_input = filterTag(tagged_user_input)

user_input_series=pd.Series(filter_tagged_user_input)
user_input_series_dtm=vect.transform(user_input_series)

predict_aspect= svc.predict(user_input_series_dtm)
print(predict_aspect)
extra_feature=get_dict_aspect(predict_aspect, most_common_aspect)
print(extra_feature)
extra_feature_dtm=DictVectorizer().fit_transform(extra_feature)
print(extra_feature_dtm)
predict_aspect

Enter a laptop review:

thái độ nhân viên tệ, thô lỗ, đồ ăn thì tệ, giá mắc
[('thái_độ', 'V'), ('nhân_viên', 'N'), ('tệ', 'A'), (',', 'F'), ('thô_lỗ', 'A'), (',', 'F'), ('đồ', 'N'), ('ăn', 'V'), ('thì', 'C'), ('tệ', 'A'), (',', 'F'), ('giá', 'N'), ('mắc', 'V')]
['thái_độ nhân_viên tệ thô_lỗ đồ ăn tệ giá mắc']
[[0 1 0 0 0 1]]
[{'AMBIENCE_GENERAL': 0, 'FOOD_PRICE': 5, 'FOOD_QUALITY': 0, 'FOOD_STYLE_OPTIONS': 0, 'RESTAURANT_GENERAL': 0, 'SERVICE_GENERAL': 5}]
  (0, 1)	5.0
  (0, 5)	5.0


array([[0, 1, 0, 0, 0, 1]])

In [140]:
#predicting weather the dectected aspect is positive or not
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_pos

0      negative
1      positive
2      negative
3      positive
4      positive
5      positive
6      positive
7      positive
8      negative
9      positive
10     positive
11     positive
12     positive
13     positive
14     positive
15     positive
16     positive
17     positive
18     positive
19     positive
20     positive
21     positive
22     positive
23     negative
24     positive
25     negative
26     negative
27     negative
28     positive
29     positive
         ...   
101    positive
102    positive
103         NaN
104    positive
105    positive
106    negative
107    positive
108    negative
109         NaN
110    negative
111         NaN
112    negative
113    positive
114         NaN
115         NaN
116    positive
117    negative
118         NaN
119    negative
120         NaN
121    negative
122    negative
123    negative
124         NaN
125    negative
126    positive
127    negative
128         NaN
129    positive
130    positive
Name: FOOD_QUALITY, Leng

/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter 

array([[1, 0, 0, 0, 0, 1]])

In [141]:
#predicting weather the dectected aspect is negative or not
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_negative = get_negative_data_frame(df_train,most_common_aspect)
y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_negative,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neg

/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/hocx3/.conda/envs/jupyter/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter 

array([[0, 0, 0, 0, 0, 0]])

In [142]:
#Finding the aspect that is positive
def aspect_sentiment():
    index_positive=[]
    for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_pos.tolist()[0])):
        if a ==1 and b==1:
            index_positive.append(i)
    
    index_negative=[]
    for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neg.tolist()[0])):
        if a ==1 and b==1:
            index_negative.append(i)
    output=[]
    if index_positive:
        for index in index_positive:
            output.append(sorted(most_common_aspect)[index]+": positive")
    if index_negative:
        for index in index_negative:
            output.append(sorted(most_common_aspect)[index]+": negative")
    return output

In [143]:
aspect_sentiment()

['AMBIENCE_GENERAL: positive', 'SERVICE_GENERAL: positive']